<br/>
<br/>
<center><font size=5 face=雅黑> 自然语言处理（实验四） </font></center>  
<br/>
<center><font size=4 face=雅黑> 隐马尔可夫模型分词 </font></center>  
<br/>
<center><font size=3 face=雅黑> 18大数据 叶腱珉 180110950330 </font></center>  
<br/>
<center><font size=3 face=雅黑> 2021 年 3 月 31 日 </font></center> 
<br/>  
<br/>
<br/>

## 练习要求

利用MSR语料计算初始状态概率向量$\pi$,状态转移概率矩阵$A$和发射概率矩阵$B$,构建分词器并进行性能评测。

## 分词器构建

### 导入需要的模块和数据

In [1]:
import time
import re
import os,sys
from math import log
from collections import Counter
import pandas as pd

sighan05 = "C:/Users/叶腱珉/Desktop/Natural Language Processing/实验二\
/第二届国际中文分词评测/icwb2-data/"
msr_train = os.path.join(sighan05,"training","msr_training.txt")

train_list = []      # 训练数据
with open(msr_train,'r') as train:
    for i in train:
        train_list.append(re.sub('\n','',i).strip())
while '' in train_list:
    train_list.remove('')

### 构建序列标注函数

In [2]:
def sentence_tagging(sentence):
    sentence_list = sentence.split()
    sentence_BMSE_list = [''.join(word) for word in [BMSE_tagging(word) for word in sentence_list]]
    return ' '.join(sentence_BMSE_list)

BMSE_dict = {1:['S'],2:['B','E']}
def BMSE_tagging(word):
    N = len(word)
    try:
        return BMSE_dict[N]
    except:
        if N < 1:
            return "ERROR"
        else:
            return ['B']+['M']*(N-2)+['E']

In [3]:
# 测试序列标注结果
msr_BMSE_list = [sentence_tagging(sentence) for sentence in train_list]

print(train_list[0],'\n')
print("\n",msr_BMSE_list[0])

“  人们  常  说  生活  是  一  部  教科书  ，  而  血  与  火  的  战争  更  是  不可多得  的  教科书  ，  她  确实  是  名副其实  的  ‘  我  的  大学  ’  。 


 S BE S S BE S S S BME S S S S S S BE S S BMME S BME S S BE S BMME S S S S BE S S


### 利用MSR语料计算初始状态概率向量 𝜋 

In [4]:
def calc_bmse_freq(BMSE_frequency):
    prob_list = {}
    sum1 = sum(BMSE_frequency.values())
    for key,value in BMSE_frequency.items():
        prob_list[key]=log(value/sum1)
    return prob_list

In [5]:
# 概率值都是取对数之后的结果
MIN_FLOAT = -3.14e100

# 利用MSR语料的标注序列，统计BMSE频数，计算初始状态概率向量
msr_prob_start = calc_bmse_freq( Counter([i[0:1] for i in msr_BMSE_list]))
# 由于初始状态只能是B或者S，故将M和E的初始状态概率设置为很小
msr_prob_start['M'] = MIN_FLOAT
msr_prob_start['E'] = MIN_FLOAT

# 初始状态概率向量
msr_prob_start

{'S': -1.1894065754192553,
 'B': -0.3629831561081316,
 'M': -3.14e+100,
 'E': -3.14e+100}

###  利用MSR语料计算状态转移概率矩阵 𝐴 

In [6]:
# 删除原序列标注句子中的空格
msr_BMSE_list1 = [re.sub(" ","",sentence) for sentence in msr_BMSE_list]

# 计算所有前后两序列组合的频率
msr_trans_list = [sentence[i:i+2] for sentence in msr_BMSE_list1 for i in range(len(sentence)-1)]
msr_trans_count = Counter(msr_trans_list)

# 统计序列频数并生成字典
msr_trans_count

Counter({'SB': 600115,
         'BE': 1039906,
         'ES': 659674,
         'SS': 427204,
         'BM': 215149,
         'ME': 215149,
         'MM': 211874,
         'EB': 594480})

In [7]:
# 统计序列频数生成字典后计算转移概率矩阵
def calc_prob_trans(BMSE_frequency):
    prob_list = {}
    BMSE_frequency.keys()
    for key,value in BMSE_frequency.items():
        if key[0] not in BMSE_frequency.keys():
            prob_list.setdefault(key[0],{})
        prob_list[key[0]][key[1]] = value
    for key in prob_list.keys():
        prob_list[key] = calc_bmse_freq(prob_list[key])
    return prob_list

# 状态转移概率矩阵
msr_prob_trans = calc_prob_trans(Counter(msr_trans_count))
msr_prob_trans

{'S': {'B': -0.5375864716182832, 'S': -0.8774461242373575},
 'B': {'E': -0.18804907187170708, 'M': -1.763603863953054},
 'E': {'S': -0.6424707470719607, 'B': -0.7465294468210316},
 'M': {'E': -0.6855070645928693, 'M': -0.7008461175870558}}

### 利用MSR语料计算发射概率矩阵 𝐵

In [8]:
# 构建单字列表
msr_words_list = [re.sub(" ","",sentence) for sentence in train_list]
msr_words_list = list(''.join(msr_words_list))
# 构建单词序列表
msr_BMSE_words_list = [re.sub(" ","",sentence) for sentence in msr_BMSE_list]
msr_BMSE_words_list = list(''.join(msr_BMSE_words_list))
# 单字和单词序列拼接组合
combineList = list(zip(msr_BMSE_words_list,msr_words_list))

In [9]:
msr_prob_emit = {}  # 发射概率矩阵
BMSE_list = list("BMSE")
for f in BMSE_list:
    # 构建二级字典
    msr_prob_emit.setdefault(f,{})
    # 筛选出某个词序的所有发射结果
    combineList_temp = [zip_i for zip_i in combineList if zip_i[0] == f]
    # 统计词序频数，并计算发射概率
    calc_emit = calc_bmse_freq(dict(Counter(combineList_temp)))
    for key,value in calc_emit.items():
        # 发射概率矩阵写入
        msr_prob_emit[key[0]][key[1]] = value

# 一级字典keys
print(msr_prob_emit.keys())
# 部分结果显示，如取序列标注为B的前五个词的发射概率结果
list(msr_prob_emit['B'].items())[:5]

dict_keys(['B', 'M', 'S', 'E'])


[('人', -4.868080392268559),
 ('生', -5.37793919851509),
 ('教', -5.731291675852302),
 ('战', -6.44178749474686),
 ('不', -4.591680504132731)]

### 构建利用msr语料计算得到初始参数的HMM模型

In [10]:
PrevStatus = {
    'B': 'ES',
    'M': 'MB',
    'S': 'SE',
    'E': 'BM'
}

In [11]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]  # tabular
    path = {}
    for y in states:  # init
        V[0][y] = start_p[y] + emit_p[y].get(obs[0], MIN_FLOAT)
        path[y] = [y]

    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            em_p = emit_p[y].get(obs[t], MIN_FLOAT)
            (prob, state) = max(
                [(V[t - 1][y0] + trans_p[y0].get(y, MIN_FLOAT) + em_p, y0) for y0 in PrevStatus[y]])
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath

    (prob, state) = max((V[len(obs) - 1][y], y) for y in 'ES')

    return (prob, path[state])

sentence = "商品和服务"
print(viterbi(sentence, "BMES", msr_prob_start, msr_prob_trans, msr_prob_emit))

(-30.294867244593323, ['B', 'E', 'S', 'B', 'E'])


In [12]:
def __cut(sentence):
    global emit_P
    prob, pos_list = viterbi(sentence, 'BMES', msr_prob_start, msr_prob_trans, msr_prob_emit)
    begin, nexti = 0, 0
    # print pos_list, sentence
    for i, char in enumerate(sentence):
        pos = pos_list[i]
        if pos == 'B':
            begin = i
        elif pos == 'E':
            yield sentence[begin:i + 1]
            nexti = i + 1
        elif pos == 'S':
            yield char
            nexti = i + 1
    if nexti < len(sentence):
        yield sentence[nexti:]
print(list(__cut(sentence)))

['商品', '和', '服务']


## 性能评测

### 加载词典

In [13]:
def load_dictionary(dict_file):
    """
    加载词库
    :return: 一个set形式的词库
    """
    fr = open(dict_file,encoding="utf-8")
    word_list = [item.strip().split("\t")[0] for item in fr]
    return set(word_list)    # 以集合形式加载，实现去重

### 性能测试

In [14]:
def to_region(segmentation: str) -> list:
    """
    将分词结果转换为区间
    :param segmentation: 商品 和 服务
    :return: [(0, 2), (2, 3), (3, 5)]
    """
    region = []
    start = 0
    for word in re.compile("\\s+").split(segmentation.strip()):
        end = start + len(word)
        region.append((start, end)) 
        start = end
    return region


def prf(gold: str, pred: str, dic) -> tuple:
    """
    计算P、R、F1
    :param gold: 标准答案文件，比如“商品 和 服务”
    :param pred: 分词结果文件，比如“商品 和服 务”
    :param dic: 词典
    :return: (P, R, F1, OOV_R, IV_R)
    """
    A_size, B_size, A_cap_B_size, OOV, IV, OOV_R, IV_R = 0, 0, 0, 0, 0, 0, 0
    with open(gold,encoding="utf-8") as gd, open(pred,encoding="utf-8") as pd:
        for g, p in zip(gd, pd):
            A, B = set(to_region(g)), set(to_region(p))
            A_size += len(A)
            B_size += len(B)
            A_cap_B_size += len(A & B)
            text = re.sub("\\s+", "", g)
            for (start, end) in A:
                word = text[start: end]
                if word in dic:
                    IV += 1
                else:
                    OOV += 1

            for (start, end) in A & B:
                word = text[start: end]
                if word in dic:
                    IV_R += 1
                else:
                    OOV_R += 1
    p, r = A_cap_B_size / B_size * 100, A_cap_B_size / A_size * 100
    return p, r, 2 * p * r / (p + r), OOV_R / OOV * 100, IV_R / IV * 100

In [15]:
msr_output = os.path.join(sighan05, 'testing', 'msr_output.txt')
msr_gold = os.path.join(sighan05, 'gold', 'msr_test_gold.utf8')

word_dict = load_dictionary("C:/Users/叶腱珉/Desktop/Natural Language Processing/\
实验二/SogouW/Freq/SogouLabDic.dic")

In [16]:
with open(msr_gold, encoding="utf-8") as test, open(msr_output, 'w', encoding="utf-8") as output:
    start = time.time()
    for line in test:
        output.write("  ".join(__cut(re.sub("\\s+", "", line))))
        output.write("\n")
    runtime = time.time()-start
print("P:%.2f R:%.2f F1:%.2f OOV-R:%.2f IV-R:%.2f" % prf(msr_gold, msr_output, word_dict))
print('runtime:{}s'.format(runtime))

P:78.03 R:80.05 F1:79.03 OOV-R:77.23 IV-R:82.81
runtime:1.3085010051727295s


### 分词结果输出保存

In [17]:
with open(msr_gold, encoding="utf-8") as test:
    pos_list_out = []
    cut_out = []
    for line in test:
        # 将每个字的标注序列输出保存，每个句子以空元素隔开
        prob, pos_list = viterbi(re.sub("\\s+", "", line), 'BMES', msr_prob_start, msr_prob_trans, msr_prob_emit)
        for w1 in pos_list:
            pos_list_out.append(w1)
        pos_list_out.append("")
        # 将每个句子拆分成单字保存
        word_split = re.sub("\\s+","",line)
        word_split = list(''.join(word_split))
        for w2 in word_split:
            cut_out.append(w2)
        cut_out.append("")

In [18]:
# 将单字和标注序列列表合成一个数据框
segmentation_out = pd.DataFrame({"word":cut_out,"tagging":pos_list_out})

In [19]:
segmentation_out

,word,tagging
0,扬,B
1,帆,E
2,远,B
3,东,E
4,做,S
...,...,...
188335,的,S
188336,对,B
188337,话,E
188338,。,S


In [20]:
# 将数据框保存为txt文件
segmentation_out.to_csv("C:/Users/叶腱珉/Desktop/Natural Language Processing\
/实验四/segmentation_result.txt",sep='\t',index=False)